In [0]:

file_location = "/FileStore/tables/riders.csv"
file_type = "csv"

infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.write.format("delta") \
.save("/delta/bronze_riders")

In [0]:

df = spark.read.format("delta") \
    .load("/delta/bronze_riders")

df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("silver_riders")

In [0]:
test = spark.sql("select * from silver_riders").show()

+-----+---------+----------+--------------------+----------+----------+----------+-----+
|  _c0|      _c1|       _c2|                 _c3|       _c4|       _c5|       _c6|  _c7|
+-----+---------+----------+--------------------+----------+----------+----------+-----+
|57257|     Mark| Mcfarland|   9928 Hunter Ranch|1982-02-01|2020-12-05|      NULL|False|
|57258|     Mark|     Davis|20036 Barrett Sum...|1963-07-28|2017-07-12|      NULL| True|
|57259|    Bryan|   Manning|    089 Sarah Square|1984-11-05|2018-08-10|      NULL| True|
|57260|  Michele|      Rowe|3157 Nicole Ferry...|1997-09-21|2016-06-03|      NULL| True|
|57261|     John|  Mckenzie|   312 Jessica Wells|2002-10-13|2016-02-01|2018-07-01| True|
|57262|     Tami|    Rivera|910 Lopez Pass Ap...|2001-06-29|2020-11-12|      NULL| True|
|57263|   Joseph|     Hodge|     411 Mccoy Haven|1993-04-30|2020-05-29|      NULL|False|
|57264|   Lauren|     Brown|  667 Rodriguez Ramp|2002-10-20|2020-05-10|      NULL|False|
|57265|Stephanie|    

In [0]:
%sql
--enabling property to update column name
ALTER TABLE silver_riders SET TBLPROPERTIES (
    'delta.minReaderVersion' = '2',
    'delta.minWriterVersion' = '5',
    'delta.columnMapping.mode' = 'name'
  )

In [0]:


%sql
--renaming default column names to reflect more descriptive, column names
ALTER TABLE silver_riders 
RENAME COLUMN _c0 TO rider_id;

ALTER TABLE silver_riders 
RENAME COLUMN _c1 TO first_name;

ALTER TABLE silver_riders 
RENAME COLUMN _c2 TO last_name;

ALTER TABLE silver_riders 
RENAME COLUMN _c3 TO address;

ALTER TABLE silver_riders 
RENAME COLUMN _c4 TO birth_date;

ALTER TABLE silver_riders 
RENAME COLUMN _c5 TO account_start_date;

ALTER TABLE silver_riders 
RENAME COLUMN _c6 TO account_end_date;

ALTER TABLE silver_riders 
RENAME COLUMN _c7 TO is_member;

In [0]:

tripsTbl = spark.table("silver_trips")
ridersTbl = spark.table("silver_riders")

from pyspark.sql.functions import *
dim_rider_df = ridersTbl.join(tripsTbl, ridersTbl.rider_id == tripsTbl.rider_id,"inner") \
    .select(ridersTbl["rider_id"].alias("rider_key"), ridersTbl["rider_id"], ridersTbl["first_name"], ridersTbl["last_name"], ridersTbl["birth_date"], tripsTbl["rideable_type"], ridersTbl["is_member"], ridersTbl["account_start_date"], ridersTbl["account_end_date"], tripsTbl["start_at"].alias("ride_start_date"), tripsTbl["ended_at"].alias("ride_end_date")).sort("rider_id") 
                
dim_rider_df.show()

dim_rider_df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_riders")

+---------+--------+----------+---------+----------+-------------+---------+------------------+----------------+-------------------+-------------------+
|rider_key|rider_id|first_name|last_name|birth_date|rideable_type|is_member|account_start_date|account_end_date|    ride_start_date|      ride_end_date|
+---------+--------+----------+---------+----------+-------------+---------+------------------+----------------+-------------------+-------------------+
|     1000|    1000|     Diana|    Clark|1989-02-13|electric_bike|     True|        2019-04-23|            NULL|2021-11-02 23:20:29|2021-11-02 23:29:16|
|     1000|    1000|     Diana|    Clark|1989-02-13|electric_bike|     True|        2019-04-23|            NULL|2021-09-21 21:53:21|2021-09-21 22:06:06|
|     1000|    1000|     Diana|    Clark|1989-02-13|electric_bike|     True|        2019-04-23|            NULL|2021-09-02 10:44:06|2021-09-02 11:10:40|
|     1000|    1000|     Diana|    Clark|1989-02-13| classic_bike|     True|      